### Notebook for the cell proportion analysis of Healthy_vs_COPD CTRL_vs_IAV data

- **Developed by**: Carlos Talavera-López Ph.D
- **Würzburg Institute for Systems Immunology - Faculty of Medicine - Julius Maximilian Universität Würzburg**
- **Created on**: 231214
- **Last modified**: 231214

### Load required packages

In [1]:
import warnings
import importlib
import numpy as np
import pertpy as pt
import altair as alt
import pandas as pd
import scanpy as sc
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt

import sccoda.datasets as scd
from sccoda.util import comp_ana as mod
from sccoda.util import data_visualization as viz
from sccoda.util import cell_composition_data as dat

To use sccoda or tasccoda please install ete3 with pip install ete3

2023-12-14 13:22:26.569466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 13:22:26.569509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 13:22:26.570493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 13:22:27.368777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Set up working environment

In [2]:
sc.settings.verbosity = 3
#sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

### Load working object

In [3]:
adata = sc.read_h5ad('../../../data/Marburg_cell_states_locked_ctl231212.raw.h5ad')
adata

AnnData object with n_obs × n_vars = 97573 × 27208
    obs: 'sex', 'age', 'ethnicity', 'PaCO2', 'donor', 'infection', 'disease', 'SMK', 'illumina_stimunr', 'bd_rhapsody', 'n_genes', 'doublet_scores', 'predicted_doublets', 'batch', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'condition', 'sample_group', 'IAV_score', 'group', 'Viral_score', 'cell_type', 'cell_states', 'leiden', 'cell_compartment', '_scvi_batch', '_scvi_labels', 'C_scANVI', 'viral_counts', 'infected_status'
    var: 'mt', 'ribo'
    uns: 'cell_states_colors', 'disease_colors', 'group_colors', 'infected_status_colors', 'infection_colors'
    obsm: 'X_scANVI', 'X_scVI', 'X_umap'

### Convert to scCODA `anndata` format

In [5]:
sccoda_model = pt.tl.sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type = "cell_level",
    generate_sample_level = True,
    cell_type_identifier = "cell_compartment",
    sample_identifier = "batch",
    covariate_obs = ["group"]
)
sccoda_data

AttributeError: module 'pertpy.tools' has no attribute 'sccoda'

### Evaluate conditions prior to modelling

In [ ]:
viz.boxplots(adata, feature_name = "group", cell_types = 'cell_compartment')
plt.show()

In [ ]:
milo.make_nhoods(adata)
milo.count_nhoods(adata, sample_col = "batch")
milo.DA_nhoods(adata, design = "~ group")

In [ ]:
milo_results = adata.uns["nhood_adata"].obs
milo_results

In [ ]:
milopy.utils.build_nhood_graph(adata)
milopy.plot.plot_nhood_graph(adata, alpha = 0.8, min_size = 0.1)

In [ ]:
milopy.utils.annotate_nhoods(adata, anno_col = 'cell_type')

In [ ]:
plt.hist(adata.uns['nhood_adata'].obs["nhood_annotation_frac"])
plt.xlabel("celltype fraction")

In [ ]:
nhood_annotation = adata.uns['nhood_adata'].obs['nhood_annotation']
nhood_annotation = nhood_annotation.cat.add_categories(['Ambiguous'])
adata.uns['nhood_adata'].obs['nhood_annotation'] = nhood_annotation

adata.uns['nhood_adata'].obs.loc[adata.uns['nhood_adata'].obs["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Ambiguous"


In [ ]:
sc.pl.violin(adata.uns['nhood_adata'], "logFC", groupby = "nhood_annotation", rotation = 90, show = False);
plt.axhline(y = 0, color = 'black', linestyle = '--');
plt.show()